In [1]:
import pandas as pd
import requests
import numpy as np
import re
from collections import Counter

In [2]:
author = pd.read_csv('../data/ica_author_data.csv')
author.shape

(13603, 12)

In [3]:
author.head()

,doi,url,year,title,journal,datePublished,authorFullName,firstName,lastName,numberOfAuthors,authorPosition,affiliation
0,10.1093/joc/jqac004,https://academic.oup.com/joc/article/72/3/297/...,2022,The Gender Divide in Wikipedia: Quantifying an...,Journal of Communication,2022-02-16,Isabelle Langrock,Isabelle,Langrock,2.0,1.0,"Annenberg School for Communication, University..."
1,10.1093/joc/jqac004,https://academic.oup.com/joc/article/72/3/297/...,2022,The Gender Divide in Wikipedia: Quantifying an...,Journal of Communication,2022-02-16,Sandra González-Bailón,Sandra,González-Bailón,2.0,2.0,"Annenberg School for Communication, University..."
2,10.1093/joc/jqac009,https://academic.oup.com/joc/article/72/3/322/...,2022,Mapping Exposure Diversity: The Divergent Effe...,Journal of Communication,2022-03-16,Pascal Jürgens,Pascal,Jürgens,2.0,1.0,"Department of Communication, Jakob-Welder-Weg ..."
3,10.1093/joc/jqac009,https://academic.oup.com/joc/article/72/3/322/...,2022,Mapping Exposure Diversity: The Divergent Effe...,Journal of Communication,2022-03-16,Birgit Stark,Birgit,Stark,2.0,2.0,"Department of Communication, Jakob-Welder-Weg ..."
4,10.1093/joc/jqac008,https://academic.oup.com/joc/article/72/3/345/...,2022,Democratic Consequences of Incidental Exposure...,Journal of Communication,2022-03-17,Andreas Nanz,Andreas,Nanz,2.0,1.0,"Department of Communication, University of Vie..."


In [4]:
firstname = list(set(author.firstName))

In [5]:
firstname = [x for x in firstname if str(x) != 'nan']
len(firstname)

2886

In [6]:
# firstname_df = pd.DataFrame(firstname)
# firstname_df.columns = ['firstname']
# firstname_df.head()

,firstname
0,Christi
1,Skye
2,Jaume
3,Eun-Ju
4,Gudbjorg


In [14]:
from genderize import Genderize
API_KEY = '486f33235ec473d0d2ceef6273dca789'
genderize = Genderize(
    user_agent='GenderizeDocs/0.0',
    api_key=API_KEY,
    timeout=5.0)
name_gender_results = genderize.get(firstname)

In [15]:
firstname_df = pd.DataFrame(name_gender_results)
firstname_df.columns = ['firstName', 'genderize', 'genderize_prob', 'genderize_basedon']
firstname_df

,firstName,genderize,genderize_prob,genderize_basedon
0,Christi,female,0.81,620
1,Skye,female,0.76,461
2,Jaume,male,0.99,5055
3,Eun-Ju,female,0.95,19
4,Gudbjorg,female,1.00,11
...,...,...,...,...
2881,Fabienne,female,0.98,21920
2882,Jiyoun,female,0.95,73
2883,Marjolein,female,0.99,1828
2884,Helmut,male,0.99,5703


In [24]:
# API_KEY = '486f33235ec473d0d2ceef6273dca789'
# genderize = []
# probability = []
# basedOn = []
# for firstName in firstname_df.firstname:
#     response = requests.get(f'https://api.genderize.io/?name={firstName}&apikey={API_KEY}')
#     j = response.json()
#     gender = j['gender']
#     prob = str(j['probability'])
#     count = str(j['count'])
#     genderize.append(gender)
#     probability.append(prob)
#     basedOn.append(count)

In [116]:
# firstname_df['genderize'] = genderize
# firstname_df['genderize_prob'] = probability
# firstname_df['genderize_basedon'] = basedOn
# firstname_df.rename(columns = {'firstname': 'firstName'}, inplace=True)

In [16]:
# predictions with low accuracy and low basedons 
low_accuracy_names_df = firstname_df[
    (firstname_df.genderize_basedon.astype(int) <= 2000) | 
    (firstname_df.genderize_prob.astype(float) <= 0.80)
]
low_accuracy_names = low_accuracy_names_df.firstName.tolist()

In [17]:
basedons = np.array(firstname_df.genderize_basedon.astype(int))
np.mean(basedons)

13460.174636174635

In [18]:
np.std(basedons)

36153.385559411705

In [19]:
np.median(basedons)

1288.0

In [20]:
firstname_df.head()

,firstName,genderize,genderize_prob,genderize_basedon
0,Christi,female,0.81,620
1,Skye,female,0.76,461
2,Jaume,male,0.99,5055
3,Eun-Ju,female,0.95,19
4,Gudbjorg,female,1.00,11


In [21]:
author.head()

,doi,url,year,title,journal,datePublished,authorFullName,firstName,lastName,numberOfAuthors,authorPosition,affiliation
0,10.1093/joc/jqac004,https://academic.oup.com/joc/article/72/3/297/...,2022,The Gender Divide in Wikipedia: Quantifying an...,Journal of Communication,2022-02-16,Isabelle Langrock,Isabelle,Langrock,2.0,1.0,"Annenberg School for Communication, University..."
1,10.1093/joc/jqac004,https://academic.oup.com/joc/article/72/3/297/...,2022,The Gender Divide in Wikipedia: Quantifying an...,Journal of Communication,2022-02-16,Sandra González-Bailón,Sandra,González-Bailón,2.0,2.0,"Annenberg School for Communication, University..."
2,10.1093/joc/jqac009,https://academic.oup.com/joc/article/72/3/322/...,2022,Mapping Exposure Diversity: The Divergent Effe...,Journal of Communication,2022-03-16,Pascal Jürgens,Pascal,Jürgens,2.0,1.0,"Department of Communication, Jakob-Welder-Weg ..."
3,10.1093/joc/jqac009,https://academic.oup.com/joc/article/72/3/322/...,2022,Mapping Exposure Diversity: The Divergent Effe...,Journal of Communication,2022-03-16,Birgit Stark,Birgit,Stark,2.0,2.0,"Department of Communication, Jakob-Welder-Weg ..."
4,10.1093/joc/jqac008,https://academic.oup.com/joc/article/72/3/345/...,2022,Democratic Consequences of Incidental Exposure...,Journal of Communication,2022-03-17,Andreas Nanz,Andreas,Nanz,2.0,1.0,"Department of Communication, University of Vie..."


## Integration

In [22]:
authorWithGender = author.merge(firstname_df, on = "firstName", how = "left")

In [23]:
authorWithGender['genderAccuracy'] = np.where(
    authorWithGender.firstName.isin(low_accuracy_names),
    'Low',
    'High'
)

In [24]:
authorWithGender.head()

,doi,url,year,title,journal,datePublished,authorFullName,firstName,lastName,numberOfAuthors,authorPosition,affiliation,genderize,genderize_prob,genderize_basedon,genderAccuracy
0,10.1093/joc/jqac004,https://academic.oup.com/joc/article/72/3/297/...,2022,The Gender Divide in Wikipedia: Quantifying an...,Journal of Communication,2022-02-16,Isabelle Langrock,Isabelle,Langrock,2.0,1.0,"Annenberg School for Communication, University...",female,0.99,89728.0,High
1,10.1093/joc/jqac004,https://academic.oup.com/joc/article/72/3/297/...,2022,The Gender Divide in Wikipedia: Quantifying an...,Journal of Communication,2022-02-16,Sandra González-Bailón,Sandra,González-Bailón,2.0,2.0,"Annenberg School for Communication, University...",female,0.98,266121.0,High
2,10.1093/joc/jqac009,https://academic.oup.com/joc/article/72/3/322/...,2022,Mapping Exposure Diversity: The Divergent Effe...,Journal of Communication,2022-03-16,Pascal Jürgens,Pascal,Jürgens,2.0,1.0,"Department of Communication, Jakob-Welder-Weg ...",male,0.99,84147.0,High
3,10.1093/joc/jqac009,https://academic.oup.com/joc/article/72/3/322/...,2022,Mapping Exposure Diversity: The Divergent Effe...,Journal of Communication,2022-03-16,Birgit Stark,Birgit,Stark,2.0,2.0,"Department of Communication, Jakob-Welder-Weg ...",female,0.98,10750.0,High
4,10.1093/joc/jqac008,https://academic.oup.com/joc/article/72/3/345/...,2022,Democratic Consequences of Incidental Exposure...,Journal of Communication,2022-03-17,Andreas Nanz,Andreas,Nanz,2.0,1.0,"Department of Communication, University of Vie...",male,0.99,71365.0,High


In [25]:
authorWithGender[authorWithGender.genderAccuracy=='Low'].shape

(3412, 16)

## Race

In [26]:
from ethnicolr import census_ln, pred_census_ln

In [27]:
race_pred = pred_census_ln(authorWithGender, 'lastName', year=2010)

2022-07-23 10:29:09.396718: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [28]:
race_pred.head()

,doi,url,year,title,journal,datePublished,authorFullName,firstName,lastName,numberOfAuthors,...,affiliation,genderize,genderize_prob,genderize_basedon,genderAccuracy,api,black,hispanic,white,race
0,10.1093/joc/jqac004,https://academic.oup.com/joc/article/72/3/297/...,2022,The Gender Divide in Wikipedia: Quantifying an...,Journal of Communication,2022-02-16,Isabelle Langrock,Isabelle,Langrock,2.0,...,"Annenberg School for Communication, University...",female,0.99,89728.0,High,0.007762,0.066429,0.030049,0.895760,white
1,10.1093/joc/jqac004,https://academic.oup.com/joc/article/72/3/297/...,2022,The Gender Divide in Wikipedia: Quantifying an...,Journal of Communication,2022-02-16,Sandra González-Bailón,Sandra,González-Bailón,2.0,...,"Annenberg School for Communication, University...",female,0.98,266121.0,High,0.010406,0.000300,0.979183,0.010111,hispanic
2,10.1093/joc/jqac009,https://academic.oup.com/joc/article/72/3/322/...,2022,Mapping Exposure Diversity: The Divergent Effe...,Journal of Communication,2022-03-16,Pascal Jürgens,Pascal,Jürgens,2.0,...,"Department of Communication, Jakob-Welder-Weg ...",male,0.99,84147.0,High,0.007112,0.088763,0.031022,0.873102,white
3,10.1093/joc/jqac009,https://academic.oup.com/joc/article/72/3/322/...,2022,Mapping Exposure Diversity: The Divergent Effe...,Journal of Communication,2022-03-16,Birgit Stark,Birgit,Stark,2.0,...,"Department of Communication, Jakob-Welder-Weg ...",female,0.98,10750.0,High,0.004433,0.043421,0.024482,0.927664,white
4,10.1093/joc/jqac008,https://academic.oup.com/joc/article/72/3/345/...,2022,Democratic Consequences of Incidental Exposure...,Journal of Communication,2022-03-17,Andreas Nanz,Andreas,Nanz,2.0,...,"Department of Communication, University of Vie...",male,0.99,71365.0,High,0.101002,0.049881,0.185628,0.663489,white


In [29]:
race_pred_sub = race_pred[['api', 'black', 'hispanic', 'white']]

In [30]:
race_pred_sub.head()

,api,black,hispanic,white
0,0.007762,0.066429,0.030049,0.895760
1,0.010406,0.000300,0.979183,0.010111
2,0.007112,0.088763,0.031022,0.873102
3,0.004433,0.043421,0.024482,0.927664
4,0.101002,0.049881,0.185628,0.663489


In [31]:
race_pred_sub.iloc[1,:].nlargest(2).values

array([0.9791829 , 0.01040568], dtype=float32)

In [32]:
# axis = 1 means row-wise
raceHighest = race_pred_sub.apply(lambda row: row.nlargest(2).values[0], axis = 1)

In [33]:
raceSecondHighest = race_pred_sub.apply(lambda row: row.nlargest(2).values[-1], axis = 1)

In [34]:
raceDiff = raceHighest - raceSecondHighest

In [35]:
raceDiff

0        0.829331
1        0.968777
2        0.784340
3        0.884243
4        0.477861
           ...   
13598    0.855996
13599    0.827481
13600    0.712756
13601    0.942900
13602    0.077081
Length: 13603, dtype: float32

In [36]:
race_pred['raceHighest'] = raceHighest
race_pred['raceSecondHighest'] = raceSecondHighest
race_pred['raceDiff'] = raceDiff

In [40]:
race_pred['racePredAccuracy'] = np.where(
    (race_pred['raceHighest'] >= 0.80) & (race_pred['raceDiff'] >= 0.30),
    'High',
    'Low'
)

In [42]:
race_pred[race_pred.racePredAccuracy == 'Low'].shape

(4666, 25)

## Affiliations

In [313]:
affs = race_pred.affiliation

In [49]:
from affiliation_parser import parse_affil
import random
import re

In [257]:
random_aff = random.choice(affs)
random_aff

'1 department of communication, university of vienna, vienna, austria'

In [258]:
random_aff = re.sub(r'\([^)]*\)', '', random_aff)
random_aff

'1 department of communication, university of vienna, vienna, austria'

In [259]:
# delete the first number
random_aff = re.sub(r"^[0-9]*\.?[0-9]+", '', random_aff).strip()
# random_aff = random_aff.strip()
random_aff

'department of communication, university of vienna, vienna, austria'

In [202]:
# https://stackoverflow.com/a/56128210
sub_patterns = ['[A-Z][a-z]* University',
                'University of [A-Z][a-z]*',
                'Ecole [A-Z][a-z]*']
pattern = '({})'.format('|'.join(sub_patterns))
matches = re.findall(pattern, random_aff)
matches

['Marquette University']

In [203]:
re.search(
    "([A-Z][^\\s,.]+[.]?\\s[(]?)*(College|University|Institute|Law School|School of|Academy)[^,\\d]*(?=,|\\d)", 
    random_aff)

<re.Match object; span=(77, 97), match='Marquette University'>

In [182]:
affs = list(set(affs))
len(affs)

9643

In [183]:
affs = [x for x in affs if str(x) != 'nan']
len(affs)

9642

In [196]:
matches = []
failed = 0
for aff in affs:
    match = re.findall(pattern, aff)
    try:
        matches.append(match[0])
    except:
        failed += 1

In [197]:
failed

1373

In [214]:
matches = []
failed = 0
failed_list = []
for aff in affs:
    aff = re.sub(r'\([^)]*\)', '', aff)
    match = re.search(
        "([A-Z][^\\s,.]+[.]?\\s[(]?)*(College|University|Institute|Law School|Academy)[^,\\d]*(?=,|\\d)", 
        aff)
    try:
        matches.append(match.group(0))
    except:
        failed_list.append(aff)
        failed += 1

In [216]:
failed_list

['2 Catherine Reiss Sloan is an attorney and an associate with Shooshan &amp; Jackson Inc., telecommunications consultants',
 '1 Joli Jensen is an associate professor of communication at the University of Tulsa.',
 "1 Kas Kalba is President of lCalba Bowen Associates, Inc. and director of the firm's multiclient study program on Japan's telecommunications services",
 '1 San Francisco State University',
 '1 William B. Gudykunst  is professor of communication at Arizona State University',
 '4 Richard A. Holmes is director of computer operations at Capital Districts Humanities Program, SUNY-Albany, N.Y. 12222',
 '1 the School of Journalism, Carleton University',
 '4 Eliana Elías Valdeavellano is the Executive Director of Minga-Perú, a health and community development nongovernmental organization in Loreto, Peru',
 '2 Richard V. Farace is Professor of Communication at Michigan State University',
 '2 Department of Psychology Old Dominion University',
 '1 Ramsey Eric Ramsey is author of The L

In [213]:
matches

['California State University',
 'Monash University',
 'Austin Peay State University',
 'Northwestern University. He formerly was Director of Human-Computer Research at Bell Communications Research. His research interests include the applications of computing and networking technology to teaching and learning',
 'The University of Texas at Austin',
 'University of Michigan',
 'University of Delaware',
 'University of Reading',
 'New Jersey Institute of Technology. Her research interests include Asynchronous Learning Networks',
 'University of Minnesota',
 'Brigham Young University',
 'Texas A&amp;M University',
 'University of California',
 'Institute for Juvenile Research',
 'New York University',
 'Millikin University',
 'University of Cologne',
 'University of Wisconsin',
 'University of California',
 'Indiana University',
 'University of Mannheim',
 'University of Pennsylvania',
 'The University of Texas at Austin ',
 'Choongnam National University in Daejun',
 'Macquarie Universit

### Try openalex data

In [217]:
author_aff = pd.read_csv('../data/interim/deprecated/author_df.csv')

In [227]:
author_aff.head()

,Title,Year,DOI,Paper URL,OpenAlex ID,Number of Authors,Author Name,Author Position,Author Position Type,OpenAlex Author ID,Author ORCID,Number of Affiliations,First Institution Name,Raw Affiliation String,First Institution ID,First Institution ROR,First Institution Type,First Institution Country Code
0,"Political Economy, Ethnocentrism and big Broth...",2022,https://doi.org/10.1080/23743670.2022.2044877,https://doi.org/10.1080/23743670.2022.2044877,W4220773556,1,Danford Zirugo,1,first,https://openalex.org/A3136918634,https://orcid.org/0000-0002-5129-1616,1.0,University of Minnesota,Hubbard School of Journalism and Mass Communic...,https://openalex.org/I130238516,https://ror.org/017zqws13,education,US
1,Mixing “Nonsense with Substance”: Negotiating ...,2022,https://doi.org/10.1080/23743670.2022.2041454,https://doi.org/10.1080/23743670.2022.2041454,W4214919791,1,Jude Nwakpoke Ogbodo,1,first,https://openalex.org/A3045374922,https://orcid.org/0000-0002-7187-785X,1.0,Ebonyi State University,"Department of Mass Communication, Ebonyi State...",https://openalex.org/I96009557,https://ror.org/01jhpwy79,education,NG
2,Towards a Journalism for Justice: A Normative ...,2022,https://doi.org/10.1080/23743670.2021.2009533,https://doi.org/10.1080/23743670.2021.2009533,W4214622661,2,Phillip Santos,1,first,https://openalex.org/A4220213478,NaN,1.0,Namibia University of Science and Technology,Namibia University of Science and Technology,https://openalex.org/I101993903,https://ror.org/03gg1ey66,education,NaN
3,Towards a Journalism for Justice: A Normative ...,2022,https://doi.org/10.1080/23743670.2021.2009533,https://doi.org/10.1080/23743670.2021.2009533,W4214622661,2,Khulekani Ndlovu,2,last,https://openalex.org/A2213311729,https://orcid.org/0000-0002-5107-9041,1.0,University of Botswana,University of Botswana,https://openalex.org/I157273133,https://ror.org/01encsj80,education,BW
4,News in the Digital Age: A Case Study of CITE ...,2022,https://doi.org/10.1080/23743670.2022.2028647,https://doi.org/10.1080/23743670.2022.2028647,W4214719854,3,N. Ndzinisa,1,first,https://openalex.org/A3111430135,https://orcid.org/0000-0002-1092-9564,1.0,"Journalism and Mass Communication, University ...","Journalism and Mass Communication, University ...",NaN,NaN,NaN,NaN


In [232]:
affnames = list(set(author_aff['First Institution Name']))
affnames = [x for x in affnames if str(x) != 'nan']
affnames = [x.lower() for x in affnames]

In [233]:
len(affnames)

14707

In [256]:
affs = race_pred.affiliation
affs = list(set(affs))
affs = [x for x in affs if str(x) != 'nan']
affs = [x.lower() for x in affs]
affs = [x.strip() for x in affs]

In [267]:
matched_dic = {}
matched = 0
failed = 0
failed_list = []
for aff in affs:
    # delete anything within ()
    aff = re.sub(r'\([^)]*\)', '', aff)
    # delete the first number
    aff = re.sub(r"^[0-9]*\.?[0-9]+", '', aff).strip()
    for x in affnames:
        if x in aff:
            matched += 1
            matched_dic[aff] = x
            break
#         else:
#             failed += 1
#             failed_list.append(aff)
#     if any(x in aff for x in affnames):
#         matched += 1
#     else:
#         failed += 1

In [272]:
pd.DataFrame(matched_dic.items()).head()

,0,1
0,catherine reiss sloan is an attorney and an as...,communication
1,patricia kearney and timothy g. plax are pro...,esso
2,"school of media, film and journalism, monash u...",media
3,"department of communication and theatre, austi...",communication
4,louis m. gomez is currently associate professo...,teaching and learning


### Trying ROR data

In [273]:
import json

In [274]:
with open('../data/raw/large/ror.json', 'r') as myfile:
    data=myfile.read()
data = json.loads(data)

In [284]:
target_str = ['university', 'college']

In [288]:
aff_names = []
for i in data:
    aff_name = i['name'].lower()
    # only want 'real universities'
    if any(x in aff_name for x in target_str):
        aff_names.append(aff_name)

In [289]:
len(aff_names)

13867

In [292]:
aff_names

['australian national university',
 'monash university',
 'university of queensland',
 'macquarie university',
 'newcastle university',
 'university of wollongong',
 'university of melbourne',
 'university of tasmania',
 'university of adelaide',
 'james cook university',
 'university of sydney',
 'flinders university',
 'rmit university',
 'la trobe university',
 'victoria university',
 'university of new england',
 'deakin university',
 'griffith university',
 'central queensland university',
 'queensland university of technology',
 'university of south australia',
 'swinburne university of technology',
 'western sydney university',
 'southern cross university',
 'bond university',
 'university of the sunshine coast',
 'warsaw university of technology',
 'charles sturt university',
 'university of canberra',
 'federation university',
 'charles darwin university',
 'university of southern queensland',
 'university mental health research institute',
 'university of turku',
 'sahlgrensk

In [293]:
matched_dic = {}
matched = 0
failed = 0
failed_list = []
for aff in affs:
    # delete anything within ()
    aff = re.sub(r'\([^)]*\)', '', aff)
    # delete the first number
    aff = re.sub(r"^[0-9]*\.?[0-9]+", '', aff).strip()
    for x in aff_names:
        if x in aff:
            matched += 1
            matched_dic[aff] = x
            break

In [297]:
len(affs)

9642

In [294]:
matched

7128

In [296]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.DataFrame(matched_dic.items()).head(30)

,0,1
0,"patricia kearney and timothy g. plax are professors in the department of speech communication, california state university, long beach. correspondence regarding this manuscript should be sent to patricia kearney at 5082 kings cross rd., westminister, ca 92683. phone: 897-5262","california state university, long beach"
1,"school of media, film and journalism, monash university, victoria, australia",monash university
2,"department of communication and theatre, austin peay state university, clarksville, tn, 37044, usa",austin peay state university
3,"louis m. gomez is currently associate professor of education and computer science at northwestern university. he formerly was director of human-computer research at bell communications research. his research interests include the applications of computing and networking technology to teaching and learning, human-computer interaction, and computer-supported cooperative work.",northwestern university
4,"school of journalism, the university of texas at austin, 300 w. dean keeton, austin, tx, 78712-1073",the university of texas at austin
5,"captain haney, who recently received the ph.d. from the university of michigan, teaches english at the united states air force academy, colorado. for three years he was a member of the air university team which presented instruction in readable writing throughout the air force",air university
6,"teresa l. thompson is assistant professor of communication at the university of delaware, newark, delaware 19711",university of delaware
7,"lecturer in it and education at the university of reading, uk. his research is largely concerned with the development of knowledge management tools for educational and other civil society projects in the uk, s.e. europe and africa. he is author of “information interventions, media development and the internet” in price, m., &amp; thompson, m. , forging peace: intervention, human rights, and the management of media space and is currently editing a collection of essays on academic collaboration and networking in s.e. europe to be published by the austrian institute of east and southeast european studies .",university of reading
8,joli jensen is an associate professor of communication at the university of tulsa.,university of tulsa
9,"department of communication studies, university of minnesota, minneapolis, mn 55455, usa",university of minnesota


In [300]:
failed_list = [x for x in affs if x not in matched_dic.keys()]

In [302]:
len(failed_list)

8656